In [1]:
pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 7.6 MB/s 
     |████████████████████████████████| 55 kB 2.2 MB/s 


In [2]:
import pandas as pd
import praw
import datetime
import numpy as np

reddit_read_only = praw.Reddit(client_id="DlcjXhOBwypg_mZEm6xlwA",         # your client id
                               client_secret="kLA5U-4G-eKi47VoCgunW1bnd2lf9Q",      # your client secret
                               user_agent="Project")        # your user agent

subreddit = reddit_read_only.subreddit("wallstreetbets")
 
# Display the name of the Subreddit
print("Display Name:", subreddit.display_name)
 
# Display the title of the Subreddit
print("Title:", subreddit.title)
 
# Display the description of the Subreddit
print("Description:", subreddit.description)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Display Name: wallstreetbets
Title: wallstreetbets
Description: Our official Twitter: [@Official_WSB](https://twitter.com/official_wsb)

---

The rules and submission guidelines are maintained on new Reddit so be sure to check them and make sure you're up to date.

* Read the [rules](https://www.reddit.com/r/wallstreetbets/about/rules)

* Read the [comment and submission guide](https://www.reddit.com/r/wallstreetbets/wiki/contentguide)

* Read the [FAQ](https://www.reddit.com/r/wallstreetbets/wiki/faq) if you're new to both wallstreetbets and trading.

---
**Join the discord**

[WSB Discord](https://discord.gg/rncPUDCJ)

**filter by flairs**

^Navigate ^WSB |^We ^recommend ^best ^daily ^DD
:--|:--     
**DD** | [All](https://ns.reddit.com/r/wallstreetbets/search?sort=new&restrict_sr=on&q=flair%3ADD) / [**Best Daily**](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3ADD&restrict_sr=on&t=day) / [Best Weekly](https://ns.reddit.com/r/wallstreetbets/search?sort=top&q=flair%3

In [ ]:
def posts_collection(stock, sort, time):
    
    posts_dict = {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Post URL": [],
              "Date": [], "Time": []
              }
    for name in stock:
        for post in reddit_read_only.subreddit("wallstreetbets").search(name,sort=sort,time_filter=time):
    # Title of each post
            posts_dict["Title"].append(post.title)
     
    # Text inside a post
            posts_dict["Post Text"].append(post.selftext)
     
    # Unique ID of each post
            posts_dict["ID"].append(post.id)
     
    # The score of a post
            posts_dict["Score"].append(post.score)
     
    # Total number of comments inside the post
            posts_dict["Total Comments"].append(post.num_comments)
     
    # URL of each post
            posts_dict["Post URL"].append(post.url)

    # Created time of each post
            posts_dict["Date"].append(post.created_utc)
    
    # Created date of each post
            dates = post.created
            posts_dict["Time"].append(datetime.datetime.fromtimestamp(dates))
                                      
    # Save posts as dataframe 
    stock_posts = pd.DataFrame(posts_dict)
    return stock_posts

In [34]:
def comments_collection(stock, sort, time):
    
    posts_dict = {"Title": [], "Post Text": [],"Score": [],
               "Post URL": [],"Time": [], "Comments":[]
              }
    comment_dict = { "Comments":[]}
    for name in stock:

        for post in reddit_read_only.subreddit("wallstreetbets").search(name,sort=sort,time_filter=time):
    # Title of each post
            posts_dict["Title"].append(post.title)
     
    # Text inside a post
            posts_dict["Post Text"].append(post.selftext)

    # The score of a post
            posts_dict["Score"].append(post.score)   
     
    # URL of each post
            posts_dict["Post URL"].append(post.url)

    # Created date of each post
            dates = post.created
            posts_dict["Time"].append(datetime.datetime.fromtimestamp(dates))

            url = "https://www.reddit.com" + post.permalink
            
            submission = reddit_read_only.submission(url=url) 
            submission .comment_sort = "best"
            submission.comment_limit = 6
            submission.comments.replace_more(limit=0)
            counter = 0

            for x in submission.comments:
              comment_dict['Comments'].append(x.body)

              if counter == 0:
                posts_dict['Comments'].append(x.body)
                counter += 1

              else:
                posts_dict["Title"].append(float('NaN'))
                posts_dict["Post Text"].append(float('NaN'))
                posts_dict["Score"].append(float('NaN'))
                posts_dict["Post URL"].append(float('NaN'))
                posts_dict['Comments'].append(x.body)
                posts_dict["Time"].append(datetime.datetime.fromtimestamp(dates))
      
    # Save posts as dataframe 
    posts_dict['Comments'].append('A')
    stock_posts = pd.DataFrame(posts_dict)
    stock_posts['Comments'].loc[stock_posts['Comments'].str.startswith('\n**User')] = float('NaN')
    df = stock_posts.copy()
    df.rename(columns={'Comments': 'Title', 'Title': 'Comments'}, inplace=True)
    df_concat = pd.concat([stock_posts,df], axis=0)
    df_concat = df_concat.drop(columns='Comments')
    df_concat = df_concat.dropna(subset=['Title'])
    return df_concat


In [ ]:
# tesla = ['tesla','TSLA']
# apple = ['apple','AAPL']
# amazon = ['amazon','AMZN']
# google = ['google','GOOG']
# meta = ['facebook','META']
# netfilx = ['netfilx','NFLX']
# time = 'month'

# # Collect posts for all six companies

# Tesla_posts = posts_collection(tesla,"new", time)
# Apple_posts = posts_collection(apple,"new", time)
# Amazon_posts = posts_collection(amazon,"new", time)
# Google_posts = posts_collection(google,"new", time)
# Meta_posts = posts_collection(meta,"new", time)
# Netfilx_posts = posts_collection(netfilx,"new", time)

In [ ]:
ids = Apple_posts["ID"]
a = Apple_posts[ids.isin(ids[ids.duplicated()])].sort_values("ID")
len(a)

10

In [42]:
tesla = ['tesla','TSLA']
apple = ['apple','AAPL']
amazon = ['amazon','AMZN']
google = ['google','GOOG']
meta = ['facebook','META']
netfilx = ['netfilx','NFLX']
time = 'month'

# Collect posts for all six companies

Tesla_posts = comments_collection(tesla,"new", time)
Apple_posts = comments_collection(apple,"new", time)
Amazon_posts = comments_collection(amazon,"new", time)
Google_posts = comments_collection(google,"new", time)
Meta_posts = comments_collection(meta,"new", time)
Netfilx_posts = comments_collection(netfilx,"new", time)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [86]:
ids = Apple_posts["Title"]
a = Apple_posts[ids.isin(ids[ids.duplicated()])].sort_values("Title")
len(a)

120

In [120]:
print('The number of data point in the Apple_posts is',len(Apple_posts))
Apple_posts_clean = Apple_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Apple_posts_clean))
ids = Apple_posts_clean["Title"]
a = Apple_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("Title")
print('The number of duplicates after cleaning up',len(a))
Apple_posts_clean.to_csv("Apple_posts_clean.csv", index=True)

The number of data point in the Apple_posts is 452
The number of data point after removing duplicates is 388
The number of duplicates after cleaning up 0


In [122]:
print('The number of data point in the Tesla_posts is',len(Tesla_posts))
Tesla_posts_clean = Tesla_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Tesla_posts_clean))
ids = Tesla_posts_clean["Title"]
a = Tesla_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("Title")
print('The number of duplicates after cleaning up',len(a))
Tesla_posts_clean.to_csv("Tesla_posts_clean.csv", index=True)

The number of data point in the Tesla_posts is 562
The number of data point after removing duplicates is 466
The number of duplicates after cleaning up 0


In [123]:
print('The number of data point in the Meta_posts is',len(Meta_posts))
Meta_posts_clean = Meta_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Meta_posts_clean))
ids = Meta_posts_clean["Title"]
a = Meta_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("Title")
print('The number of duplicates after cleaning up',len(a))
Meta_posts_clean.to_csv("Meta_posts_clean.csv", index=True)

The number of data point in the Meta_posts is 363
The number of data point after removing duplicates is 305
The number of duplicates after cleaning up 0


In [17]:
print('The number of data point in the Amazon_posts is',len(Amazon_posts))
Amazon_posts_clean = Amazon_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Amazon_posts_clean))
ids = Amazon_posts_clean["Title"]
a = Amazon_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("Title")
print('The number of duplicates after cleaning up',len(a))
Amazon_posts_clean.to_csv("Amazon_posts_clean.csv", index=True)

The number of data point in the Amazon_posts is 505
The number of data point after removing duplicates is 386
The number of duplicates after cleaning up 0


In [46]:
print('The number of data point in the Google_posts is',len(Google_posts))
Google_posts_clean = Google_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Google_posts_clean))
ids = Google_posts_clean["Title"]
a = Google_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("Title")
print('The number of duplicates after cleaning up',len(a))
Google_posts_clean.to_csv("Google_posts_clean.csv", index=True)

The number of data point in the Google_posts is 361
The number of data point after removing duplicates is 354
The number of duplicates after cleaning up 0


In [14]:
print('The number of data point in the Netflix_posts is',len(Netfilx_posts))
Netfilx_posts_clean = Netfilx_posts.drop_duplicates(subset=['Title'])
print('The number of data point after removing duplicates is',len(Netfilx_posts_clean))
ids = Netfilx_posts_clean["Title"]
a = Netfilx_posts_clean[ids.isin(ids[ids.duplicated()])].sort_values("Title")
print('The number of duplicates after cleaning up',len(a))
Netfilx_posts_clean.to_csv("Netfilx_posts_clean.csv", index=True)

The number of data point in the Netflix_posts is 184
The number of data point after removing duplicates is 180
The number of duplicates after cleaning up 0
